In [1]:
# pip install faker
# pip install psycopg2-binary

In [2]:
import psycopg2  # mengimpor modul psycopg2 untuk koneksi ke database PostgreSQL
import random  # mengimpor modul random untuk menghasilkan nilai acak
import time  # mengimpor modul time untuk menambahkan penundaan waktu dalam program
from faker import Faker  # mengimpor modul Faker untuk membuat data palsu

In [3]:
fake = Faker('id_ID')  # inisialisasi objek Faker karena kita ingin menggunakan data indonesia maka menggunakan id_ID

# Koneksi ke database PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="postgres",
    user="postgres",
    password="admin"
)

cur = conn.cursor()  # kursor database yang digunakan untuk mengeksekusi perintah SQL pada database

In [5]:
# mendefine list location_id yang akan digunakan untuk mengenerate lokasi user dan seller
location_id = [3171,3172,3173,3174,3175,3573,3578,3471,3273,1371,1375,6471,6472,7371,5171]

**Insert Data Faker User Seller**

In [82]:
for i in range(100):
    name = fake.name()
    contact = fake.phone_number()
    location = random.choice(location_id)
    user = (i+1, name, contact, location)
    cur.execute("INSERT INTO dim_user_seller(user_id, name, contact, location_id) VALUES (%s, %s, %s, %s)", user)
    conn.commit()


In [83]:
for i in range(100):
    name = fake.name()
    contact = fake.phone_number()
    location = random.choice(location_id)
    user = (i+1, name, contact, location)
    cur.execute("INSERT INTO dim_user_buyyer(user_id, name, contact, location_id) VALUES (%s, %s, %s, %s)", user)
    conn.commit()


In [87]:
titles = ["mobil bekas", "jual mobil", "jual mobil bekas", "jual beli mobil", "mobil second", "mobil", "jual beli mobil bekas", "mobil bekas jakarta", "mobil bekas murah", "mobil murah", "jual mobil second", "mobil bekas berkualitas", "jual mobil murah", "jual mobil jakarta", "toyota", "mobil dijual", "mobil bekas surabaya", "honda", "otomotif", "mobil bekas dijual", "mobil baru", "jual mobil mewah", "jakarta", "indonesia", "jual mobil surabaya", "beli mobil", "jual mobil bekas jakarta", "mobkas", "daihatsu", "mobil mewah"]


In [90]:
cursor = conn.cursor()
for i in range(1000):
    ad_id = i+1
    product_id = fake.random_int(min=1, max=50)
    user_id = fake.random_int(min=1, max=100)
    title = fake.random_element(titles)
    description = fake.paragraph(nb_sentences=5)
    price_sell = fake.random_int(min=100000000, max=400000000)
    date_created = fake.date_time_between(start_date='-1y', end_date='now')
    query = f"INSERT INTO public.fact_advertisement VALUES ({ad_id}, {product_id}, {user_id}, '{title}', '{description}', {price_sell}, '{date_created}')"
    cursor.execute(query)

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

In [100]:
cursor = conn.cursor()
for i in range(300):
    bid_buyer_id = i+1
    user_id_buyer = fake.random_int(min=1, max=70)
    product_id = fake.random_int(min=1, max=50)
    bid_price = fake.random_int(min=100000000, max=400000000)
    date_created = fake.date_time_between(start_date='-1y', end_date='now')
    query = f"INSERT INTO public.dim_bid_buyer VALUES ({bid_buyer_id}, {user_id_buyer}, {product_id}, {bid_price}, '{date_created}')"
    cursor.execute(query)

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

In [112]:
location_interest = random.choice(location_id) 

In [6]:
# Mengisi tabel "fact_interest" dengan 1000 baris data palsu
for i in range(1000):
    interest_id = i + 1
    user_id_buyer = random.randint(1, 70)  # Angka 1-70 sebagai user_id_buyer
    product_id = random.randint(1, 50)  # Angka 1-50 sebagai product_id
    location_interest = random.choice(location_id)  # Memilih lokasi secara acak
    date_created = fake.date_time_between(start_date='-1y', end_date='now')  # Waktu antara 1 tahun yang lalu dan sekarang
    # Menyimpan data palsu ke database
    cur.execute("INSERT INTO public.fact_interest (interest_id, user_id_buyer, product_id, location_id, date_created) VALUES (%s, %s, %s, %s, %s)", (interest_id, user_id_buyer, product_id, location_interest, date_created))

# Menyimpan perubahan ke database
conn.commit()


In [106]:
conn.rollback()  # rollback the current transaction
cur = conn.cursor()  # create a new cursor object

**Langkah #4 - Creating Transactional Query**

In [ ]:
## **Mencari Kendaraan Tahun >= 2015**
"""
select dp.product_id, dp.brand, dp.model, dp.year, fa.price_sell
from fact_advertisement fa 
join dim_product dp using(product_id)
where dp.year >= 2015
"""

## 
"""
insert into dim_bid_buyer (bid_buyer_id ,user_id_buyer, 
product_id, bid_price, date_created)
values (301, 29,10,188888888,'2022-05-15 16:00:25.000')
"""

## Contoh: Mobil yang dijual oleh akun oni putra
"""
select dp.product_id , dp.brand , dp.model , dp."year" ,
fa.price_sell as price, fa.date_created as date_post
from fact_advertisement fa
join dim_product dp using (product_id)
join dim_user_seller dus  using (user_id)
where dus."name" = 'Oni Putra'
order by fa.date_created desc
"""

## Mencari mobil bekas yang termurah berdasarkan keyword
"""
select dp.product_id , dp.brand , dp.model , dp."year" ,
fa.price_sell as price, fa.date_created as date_post
from fact_advertisement fa
join dim_product dp using (product_id)
join dim_user_seller dus  using (user_id)
where dp.model = 'Toyota Agya'
order by fa.price_sell asc  
"""

## Mencari mobil bekas yang terdekat berdasarkan sebuah id kota, jarak terdekat dihitung berdasarkan latitude longitude
## dengan catatan kota referensi tersebut dimasukan sehingga jarak yang dihitung adalah antar kota tersebut 
"""WITH city_location AS (
  ## CTE untuk menyimpan latitude dan longitude kota referensi
  SELECT longitude, latitude
  FROM dim_location
  WHERE location_id = 3173 ## id kota referensi
)
SELECT dp.product_id, dp.brand, dp.model, dp."year", fa.price_sell as price, 
       fa.date_created as date_post, dl.longitude, dl.latitude,
       SQRT(POW(dl.latitude - cl.latitude, 2) + POW(dl.longitude - cl.longitude, 2)) as jarak
FROM fact_advertisement fa
JOIN dim_product dp USING (product_id)
JOIN dim_user_seller dus USING (user_id)
JOIN dim_location dl ON dus.location_id = dl.location_id 
CROSS JOIN city_location cl ## melakukan cross join dengan CTE city_location
WHERE dus.location_id <> 3173 ## hanya mencari mobil bekas di luar kota referensi
ORDER BY jarak ASC ## mengurutkan hasil berdasarkan jarak dan harga jual
"""

**Langkah #5 - Creating Analytical Query**

In [ ]:
## Test 1
## Ranking popularitas model mobil berdasarkan jumlah bid

"""
with product_bid as (
select product_id, count(product_id) as count_bid_product from dim_bid_buyer dbb 
group by product_id order by count(product_id) desc 
),
product_sell as 
(
select product_id , count(product_id) as count_product_sell from fact_advertisement
group by product_id 
)
select product_id, count_bid_product, count_product_sell
from product_bid
join product_sell using (product_id)
order by count_bid_product desc
"""

## Test 2
## Membandingkan harga mobil berdasarkan harga rata-rata per kota
"""
SELECT dl.city, dp.brand, dp.model, dp."year",
       AVG(fa.price_sell) OVER (PARTITION BY dl.city) AS avg_price
FROM fact_advertisement fa
JOIN dim_product dp USING (product_id)
JOIN dim_user_seller dus USING (user_id)
JOIN dim_location dl ON dus.location_id = dl.location_id
order by dp."year" desc
 """

 
## Test 3
## Dari penawaran suatu model mobil, cari perbandingan tanggal user melakukan bid 
## dengan bid selanjutnya beserta harga tawar yang diberikan
"""
with cte as 
(
SELECT dp.model, dbb.user_id_buyer, dbb.date_created AS bid_date, 
       LEAD(dbb.date_created) OVER (PARTITION BY dp.model, dbb.user_id_buyer ORDER BY dbb.date_created) AS next_bid_date,
       dbb.bid_price, 
       LEAD(dbb.bid_price) OVER (PARTITION BY dp.model, dbb.user_id_buyer ORDER BY dbb.date_created) AS next_bid_price
FROM dim_bid_buyer dbb
JOIN dim_product dp ON dbb.product_id = dp.product_id
WHERE dp.model = 'Toyota Yaris' 
)
select * from cte 
where next_bid_date is not null and next_bid_price is not null
ORDER BY cte.user_id_buyer, cte.bid_date;
"""

## Test 4
## Membandingkan persentase perbedaan rata-rata harga mobil berdasarkan modelnya 
## dan rata-rata harga bid yang ditawarkan oleh customer pada 6 bulan terakhir 

""" WITH cte_avg_price AS (
## Menghitung Harga Rata-Rata mobil
SELECT dp.brand, dp.model, AVG(fa.price_sell) AS avg_price
  FROM fact_advertisement fa
  JOIN dim_product dp USING (product_id)
  GROUP BY dp.brand, dp.model
), 
cte_avg_bid AS (
SELECT dp.brand, dp.model, 
AVG(dbb.bid_price) AS avg_bid_6month
FROM dim_bid_buyer dbb
JOIN dim_product dp ON dbb.product_id = dp.product_id
WHERE dbb.date_created >= NOW() - INTERVAL '6 months'
group by dp.brand, dp.model
)
select 
cte_avg_price.brand, 
cte_avg_price.model, 
ROUND(cte_avg_price.avg_price,2) as avg_price,
ROUND(cte_avg_bid.avg_bid_6month,2) as avg_bid_6month,
ROUND(cte_avg_price.avg_price - cte_avg_bid.avg_bid_6month, 2) AS difference,
ROUND(((cte_avg_price.avg_price - cte_avg_bid.avg_bid_6month) / cte_avg_price.avg_price ) * 100, 2) AS difference_percent
from cte_avg_price
join cte_avg_bid using (model)
order by cte_avg_price.brand
"""

## Test 5
## Membuat  rata-rata harga bid sebuah merk dan model mobil selama 6 bulan terakhir
"""
SELECT dp.brand, dp.model,
       AVG(CASE WHEN dbb.date_created BETWEEN NOW() - INTERVAL '6 months' AND NOW() 
                THEN dbb.bid_price ELSE NULL END) AS m_min_1,
       AVG(CASE WHEN dbb.date_created BETWEEN NOW() - INTERVAL '6 months' - INTERVAL '1 month' AND NOW() - INTERVAL '1 month'
                THEN dbb.bid_price ELSE NULL END) AS m_min_2,
       AVG(CASE WHEN dbb.date_created BETWEEN NOW() - INTERVAL '6 months' - INTERVAL '2 months' AND NOW() - INTERVAL '2 months'
                THEN dbb.bid_price ELSE NULL END) AS m_min_3,
       AVG(CASE WHEN dbb.date_created BETWEEN NOW() - INTERVAL '6 months' - INTERVAL '3 months' AND NOW() - INTERVAL '3 months'
                THEN dbb.bid_price ELSE NULL END) AS m_min_4,
       AVG(CASE WHEN dbb.date_created BETWEEN NOW() - INTERVAL '6 months' - INTERVAL '4 months' AND NOW() - INTERVAL '4 months'
                THEN dbb.bid_price ELSE NULL END) AS m_min_5,
       AVG(CASE WHEN dbb.date_created BETWEEN NOW() - INTERVAL '6 months' - INTERVAL '5 months' AND NOW() - INTERVAL '5 months'
                THEN dbb.bid_price ELSE NULL END) AS m_min_6
FROM dim_bid_buyer dbb
JOIN dim_product dp ON dbb.product_id = dp.product_id
WHERE dbb.date_created >= NOW() - INTERVAL '6 months'
GROUP BY dp.brand, dp.model
"""